[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Machine Learning Methods

## Supervised Learning - Classification - Confusion Matrix and Cross Validation - Exercise

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.001 | 20/03/2024 | Royi Avital | Added SSL bypass in case of failing SSL Certificate                |
| 1.0.000 | 13/03/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0034ConfMatCrossValidation.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import fetch_openml
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.svm import LinearSVC, SVC

# Miscellaneous
from platform import python_version
import random

# Typing
from typing import Callable, Dict, List, Optional, Set, Tuple, Union

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

In [ ]:
# Courses Packages

from DataVisualization import PlotConfusionMatrix, PlotLabelsHistogram, PlotMnistImages

In [ ]:
# General Auxiliary Functions


## Exercise - Cross Validation with the SVM

In this exercise we'll apply the Cross Validation manually to find the optimal `C` parameter for the SVM Model.  
Instead of using `cross_val_predict()` we'll do a manual loop on the folds and average the score.

1. Load the [MNIST Data set](https://en.wikipedia.org/wiki/MNIST_database) using `fetch_openml()`.
2. Split the data using Stratified K Fold.
3. For each model (Parameterized by `C`):
    - Train model on the train sub set.
    - Score model on the test sub set.
4. Plot the score per model.
5. Plot the Confusion Matrix of the best model on the training data.

* <font color='brown'>(**#**)</font> Make sure to chose small number of models and folds at the beginning to measure run time and scale accordingly. 
* <font color='brown'>(**#**)</font> We'll use `LinearSVC` class which optimized `SVC` with kernel `linear` as it fits for larger data sets.  
* <font color='brown'>(**#**)</font> You may and should use the functions in the `Auxiliary Functions` section.

In [ ]:
# Parameters

numSamples  = 10_000
numImg = 3

maxItr = 5000 #<! For the LinearSVC model

#===========================Fill This===========================#
# 1. Set the number of folds.
# 1. Set the values of the `C` parameter grid.
numFold = 5
lC = list(np.linspace(0.0005, 1.5, 15))
#===============================================================#

## Generate / Load Data

The _MNIST_ database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits.  
The MNIST data is a well known data set in Machine Learning, basically it is the _Hello World_ of ML.

The original black and white images from NIST were normalized to fit into a `28x28` pixel bounding box and anti aliased.

* <font color='brown'>(**#**)</font> There is an extended version called [EMNIST](https://arxiv.org/abs/1702.05373).


In [ ]:
# Failing SSL Certificate
# In case `fetch_openml()` fails with SSL Certificate issue, run this.
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Load Data 

#===========================Fill This===========================#
# 1. Load the MNIST Data using `fetch_openml`.
# !! Use the option `parser = auto`.
mX, vY = fetch_openml('mnist_784', version = 1, return_X_y = True, as_frame = False, parser = 'auto')
vY = vY.astype(np.int_) #<! The labels are strings, convert to integer
#===============================================================#

# The data has many samples, for fast run time we'll sub sample it.

vSampleIdx = np.random.choice(mX.shape[0], numSamples)
mX = mX[vSampleIdx, :]
vY = vY[vSampleIdx]

print(f'The features data shape: {mX.shape}')
print(f'The labels data shape: {vY.shape}')
print(f'The unique values of the labels: {np.unique(vY)}')

In [ ]:
# Pre Process Data
# Scaling the data values.

# The image is in the range {0, 1, ..., 255}.
# We scale it into [0, 1].

#===========================Fill This===========================#
# 1. Scale the features value into the [0, 1] range.
# !! Try implementing it in place.

mX = mX / 255.0

#===============================================================#

### Plot Data

In [ ]:
# Plot the Data

hF = PlotMnistImages(mX, vY, numImg)

### Distribution of Labels

When dealing with classification, it is important to know the balance between the labels within the data set.

In [ ]:
# Distribution of Labels

hA = PlotLabelsHistogram(vY)

* <font color='red'>(**?**)</font> Looking at the _histogram_ of labels, Is the data balanced?

## Cross Validation

The _Cross Validation_ process has 2 main objectives:

1. Estimate the real world performance and its stability.   
2. Optimize the model _Hyper Parameters_. 

### Cross Validation for Hyper Parameter Optimization

We can also use the _Cross Validation_ approach to search for the best _Hype Parameter_.  
The idea is iterating through the data and measure the score we care about.  
The hyper parameter which maximize the score will be used for the production model.

* <font color='red'>(**?**)</font> What kind of a problem is this? Binary Class or Multi Class?
* <font color='red'>(**?**)</font> What kind of strategy will be used? Advise documentation.
* <font color='brown'>(**#**)</font> When using `LinearSVC`:
    *   If #Samples > #Features -> Set `dual = False`.
    *   If #Samples < #Features -> Set `dual = True` (Default).
* <font color='brown'>(**#**)</font> If you experience converging issues with `LinearSVC` use `SVC`.

In [ ]:
# Cross Validation for the C parameter
numC = len(lC)
mACC = np.zeros(shape = (numFold, numC)) #<! Accuracy per Fold and Model

oStrCv = StratifiedKFold(n_splits = numFold, random_state = seedNum, shuffle = True)

for ii, (vTrainIdx, vTestIdx) in enumerate(oStrCv.split(mX, vY)):
    print(f'Working on Fold #{(ii + 1):02d} Out of {numFold} Folds')
    #===========================Fill This===========================#
    # Setting the Train / Test split
    mXTrain = mX[vTrainIdx, :]
    vYTrain = vY[vTrainIdx]
    mXTest  = mX[vTestIdx, :]
    vYTest  = vY[vTestIdx]
    #===============================================================#
    for jj, C in enumerate(lC):
        # print(f'Working on Model #{(jj + 1):02d} Out of {numC} Models with C = {C:0.4f}')
        #===========================Fill This===========================#
        # Set the model, train, score
        # Set `max_iter = maxItr`
        # Set `dual = False`
        oSvmCls     = LinearSVC(C = C, max_iter = maxItr, dual = False)
        # oSvmCls     = SVC(C = C)
        oSvmCls     = oSvmCls.fit(mXTrain, vYTrain)
        accScore    = oSvmCls.score(mXTest, vYTest)
        #===============================================================#
        print(f'Finished Evaluating on Model #{(jj + 1):02d} Out of {numC} Models with C = {C:0.4f} with score: {accScore:0.2f}')
        mACC[ii, jj] = accScore

* <font color='red'>(**?**)</font> How can we accelerate the above calculation?   
  Think about dependency between the scores, does it exist?

In [ ]:
# Model Score

#===========================Fill This===========================#
# 1. Calculate the score per model (Reduction).
# !! Average over the different folds.

vAvgAcc = np.mean(mACC, axis = 0) #<! Accuracy
#===============================================================#

* <font color='red'>(**?**)</font> In the above we used the mean as the reduction operator of many results into one. Can you think on other operators?
* <font color='blue'>(**!**)</font> Try using a different reduction method and see results.

In [ ]:
# Plot Results

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA.plot(lC, vAvgAcc)
hA.scatter(lC, vAvgAcc, s = 100)
hA.set_title(f'Accuracy Score as a Function of C - Average of {numFold} Folds')
hA.set_xlabel('C')
hA.set_ylabel('Accuracy')
hA.set_xticks(lC)
hA.grid()

* <font color='red'>(**?**)</font> What range would you choose to do a fine tune over?

## Confusion Matrix

The confusion matrix is almost the whole story for classification problems.  

Train the model with the best parameter on the whole data and plot the _Confusion Matrix_.

In [ ]:
# Optimal Parameter

#===========================Fill This===========================#
# Extract the optimal C
# Look at `np.argmax()`

optC = lC[np.argmax(vAvgAcc)] #<! Optimal `C` value 
#===============================================================#

print(f'The optimal C value is C = {optC}')

In [ ]:
# Plot the Confusion Matrix 

#===========================Fill This===========================#
# 1. Build the SVC model with the best parameter.
# 2. Fit & Predict using the model.
# 3. Calculate the accuracy score.
oSvmCls = LinearSVC(C = optC, dual = 'auto') #<! The model object
oSvmCls = oSvmCls.fit(mX, vY) #<! Fit to data
vYPred = oSvmCls.predict(mX) #<! Predict on the data
dScore = {'Accuracy': np.mean(vYPred == vY)} #<! Dictionary with the `Accuracy` as its key
#===============================================================#

hF, hA = plt.subplots(figsize = (8, 8))

PlotConfusionMatrix(vY, vYPred, normMethod = 'true', hA = hA, dScore = dScore, valFormat = '0.1%') #<! The accuracy should be >= than above!

* <font color='red'>(**?**)</font> Is the accuracy above higher or smaller than the one on the _cross validation_? Why?
* <font color='blue'>(**!**)</font> Run the above using `SVC()` instead of `LinearSVC()`.